In [59]:
import re
import chardet
from razdel import sentenize
from simalign import SentenceAligner

RU_FILE = "belye_nochi_ru.txt"
EN_FILE = "belye_nochi_en.txt"

def detect_encoding(path):
    with open(path, "rb") as f:
        return chardet.detect(f.read())["encoding"]

def read_text(path):
    enc = detect_encoding(path)
    with open(path, "r", encoding=enc, errors="ignore") as f:
        return f.read()

def tokenize_ru(text):
    return [s.text for s in sentenize(text)]

def tokenize_en(text):
    sents = re.split(r'(?<=[\.!?])\s+', text.strip())
    return [s for s in sents if s]

def run_simalign(rus_sents, eng_sents, model_name="distilbert-base-multilingual-cased", output_file=None):
    print("Инициализация SimAlign...")

    # Подбираем корректный слой для distilbert, у неё всего 7 слоев (0..6)
    layer_for_model = None
    if model_name.startswith("distilbert"):
        layer_for_model = 6  # последний слой

    try:
        if layer_for_model is not None:
            aligner = SentenceAligner(model=model_name,
                                      token_type="bpe",
                                      matching_methods="mwmf",
                                      layer=layer_for_model)
        else:
            aligner = SentenceAligner(model=model_name,
                                      token_type="bpe",
                                      matching_methods="mwmf")
    except Exception as e:
        print(f"Ошибка при создании aligner с методом 'mwmf': {e}")
        print("Попытка инициализации без параметра matching_methods...")
        try:
            if layer_for_model is not None:
                aligner = SentenceAligner(model=model_name,
                                          token_type="bpe",
                                          layer=layer_for_model)
            else:
                aligner = SentenceAligner(model=model_name,
                                          token_type="bpe")
        except Exception as e2:
            print(f"Ошибка при создании aligner без matching_methods: {e2}")
            raise e2

    print("SimAlign инициализирован успешно!")

    flags = []
    results = []

    print(f"Начинаем обработку {len(rus_sents[:10])} предложений...")

    for i, (r, e) in enumerate(zip(rus_sents[:10], eng_sents[:10])):
        print(f"Обработка пары {i+1}...")
        r_tokens = r.split()
        e_tokens = e.split()

        try:
            res = aligner.get_word_aligns(r_tokens, e_tokens)
            print(f"Ключи в результате: {list(res.keys())}")
            if not res:
                print("Пустой результат выравнивания")
                aligned_pairs = []
            else:
                method_name = list(res.keys())[0]
                print(f"Используем метод: {method_name}")
                aligned_pairs = res.get(method_name, [])

            has_align = bool(aligned_pairs)
            flags.append(has_align)
            results.append({"ru_tokens": r_tokens,
                            "en_tokens": e_tokens,
                            "alignment": aligned_pairs})
        except Exception as e:
            print(f"Ошибка при выравнивании пары {i+1}: {e}")
            flags.append(False)
            results.append({"ru_tokens": r_tokens,
                            "en_tokens": e_tokens,
                            "alignment": []})

    return flags, results


def main():
    print("Чтение файлов...")
    ru_text = read_text(RU_FILE)
    en_text = read_text(EN_FILE)

    print("Токенизация...")
    rus_sents = tokenize_ru(ru_text)
    eng_sents = tokenize_en(en_text)

    n = min(len(rus_sents), len(eng_sents))
    rus_sents = rus_sents[:n]
    eng_sents = eng_sents[:n]

    print(f"Обработано предложений: {n}")

    try:
        import simalign
        version = getattr(simalign, '__version__', 'Неизвестно')
        print(f"Версия SimAlign: {version}")
    except Exception as e:
        print(f"Ошибка при проверке версии: {e}")

    print("Проверка методов SimAlign...")
    try:
        test_aligner = SentenceAligner(model="bert-base-multilingual-cased")

        import inspect
        try:
            sig = inspect.signature(SentenceAligner.__init__)
            if 'matching_methods' in sig.parameters:
                param = sig.parameters['matching_methods']
                print(f"Параметр matching_methods: {param.default}")
            else:
                print("matching_methods не найден в сигнатуре")
        except Exception as e:
            print(f"Ошибка при проверке сигнатуры: {e}")

        print("Тестовый запуск...")
        ru_sample = "Это тестовое предложение"
        en_sample = "This is a test sentence"
        result = test_aligner.get_word_aligns(ru_sample.split(), en_sample.split())
        print(f"Доступные методы в результате: {list(result.keys())}")

    except Exception as e:
        print(f"Ошибка при проверке методов: {e}")

    print("Запуск основного выравнивания...")
    try:
        flags, aligned = run_simalign(rus_sents, eng_sents)
        total_aligned = sum(flags)
        print(f"SimAlign нашёл выравнивание в {total_aligned}/{n} предложениях\n")

        print("Примеры выравниваний (первые 5):\n")
        count = 0
        for i, item in enumerate(aligned):
            if flags[i]:
                print(f"#Пара {i+1}:")
                print("RU:", rus_sents[i])
                print("EN:", eng_sents[i])
                print("Alignment:", item['alignment'])
                print()
                count += 1
                if count >= 5:
                    break
    except Exception as e:
        print(f"Error running SimAlign: {e}")
        try:
            import simalign
            print(f"SimAlign version: {simalign.__version__ if hasattr(simalign, '__version__') else 'Unknown'}")
        except:
            print("Couldn't determine SimAlign version")

if __name__ == "__main__":
    main()

Чтение файлов...
Токенизация...
Обработано предложений: 1064
Версия SimAlign: Неизвестно
Проверка методов SimAlign...


2025-05-17 02:09:33,218 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: bert-base-multilingual-cased


Параметр matching_methods: mai
Тестовый запуск...
Доступные методы в результате: ['mwmf', 'inter', 'itermax']
Запуск основного выравнивания...
Инициализация SimAlign...
Ошибка при создании aligner с методом 'mwmf': 'w'
Попытка инициализации без параметра matching_methods...


2025-05-17 02:09:33,987 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: distilbert-base-multilingual-cased


SimAlign инициализирован успешно!
Начинаем обработку 10 предложений...
Обработка пары 1...
Ключи в результате: ['mwmf', 'inter', 'itermax']
Используем метод: mwmf
Обработка пары 2...
Ключи в результате: ['mwmf', 'inter', 'itermax']
Используем метод: mwmf
Обработка пары 3...
Ключи в результате: ['mwmf', 'inter', 'itermax']
Используем метод: mwmf
Обработка пары 4...
Ключи в результате: ['mwmf', 'inter', 'itermax']
Используем метод: mwmf
Обработка пары 5...
Ключи в результате: ['mwmf', 'inter', 'itermax']
Используем метод: mwmf
Обработка пары 6...
Ключи в результате: ['mwmf', 'inter', 'itermax']
Используем метод: mwmf
Обработка пары 7...
Ключи в результате: ['mwmf', 'inter', 'itermax']
Используем метод: mwmf
Обработка пары 8...
Ключи в результате: ['mwmf', 'inter', 'itermax']
Используем метод: mwmf
Обработка пары 9...
Ключи в результате: ['mwmf', 'inter', 'itermax']
Используем метод: mwmf
Обработка пары 10...
Ключи в результате: ['mwmf', 'inter', 'itermax']
Используем метод: mwmf
SimAlign

In [60]:
import re
import chardet
from razdel import sentenize
from simalign import SentenceAligner
import numpy as np
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import torch

RU_FILE = "belye_nochi_ru.txt"
EN_FILE = "belye_nochi_en.txt"
OUTPUT_FILE = "alignment_results.txt"

def detect_encoding(path):
    with open(path, "rb") as f:
        return chardet.detect(f.read())["encoding"]

def read_text(path):
    enc = detect_encoding(path)
    with open(path, "r", encoding=enc, errors="ignore") as f:
        return f.read()

def tokenize_ru(text):
    return [s.text for s in sentenize(text)]

def tokenize_en(text):
    sents = re.split(r'(?<=[\.!?])\s+', text.strip())
    return [s for s in sents if s]

def get_embeddings(tokens, model, tokenizer, device='cpu'):
    inputs = tokenizer(tokens, return_tensors="pt", padding=True, truncation=True, is_split_into_words=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
    return embeddings

def run_simalign(rus_sents, eng_sents, model_name="distilbert-base-multilingual-cased", output_file=None):
    print("Инициализация SimAlign...")
    methods = ["inter", "itermax"]
    try:
        aligner = SentenceAligner(model=model_name, token_type="bpe", matching_methods=methods)
    except Exception as e:
        print(f"Ошибка при создании aligner: {e}")
        print("Попытка создания с настройками по умолчанию...")
        aligner = SentenceAligner(model=model_name, token_type="bpe", layer=5)
        methods = ["inter"]

    print("SimAlign готов!")

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    model.eval()

    flags = {method: [] for method in methods}
    results = {method: [] for method in methods}
    cosine_scores = {method: [] for method in methods}

    print(f"Обработка {len(rus_sents)} предложений...")

    if output_file:
        f_out = open(output_file, 'w', encoding='utf-8')
        f_out.write(f"Всего предложений для обработки: {len(rus_sents)}\n\n")

    for i, (r, e) in enumerate(zip(rus_sents, eng_sents)):
        if i % 50 == 0:
            print(f"Обработано {i}/{len(rus_sents)} предложений...")

        r_tokens = r.split()
        e_tokens = e.split()

        try:
            res = aligner.get_word_aligns(r_tokens, e_tokens)
            for method in methods:
                if method not in res:
                    print(f"Предупреждение: метод '{method}' отсутствует в результатах.")
                    flags[method].append(False)
                    results[method].append({"ru_tokens": r_tokens, "en_tokens": e_tokens, "alignment": []})
                    cosine_scores[method].append(0.0)
                    continue

                aligned_pairs = res[method]
                has_align = bool(aligned_pairs)
                flags[method].append(has_align)
                results[method].append({"ru_tokens": r_tokens, "en_tokens": e_tokens, "alignment": aligned_pairs})

                if has_align:
                    ru_aligned = [r_tokens[idx] for idx, _ in aligned_pairs if idx < len(r_tokens)]
                    en_aligned = [e_tokens[idx] for _, idx in aligned_pairs if idx < len(e_tokens)]
                    if ru_aligned and en_aligned:
                        ru_embeds = get_embeddings(ru_aligned, model, tokenizer, device)
                        en_embeds = get_embeddings(en_aligned, model, tokenizer, device)
                        cos_sim = cosine_similarity(ru_embeds, en_embeds).diagonal().mean()
                        cosine_scores[method].append(cos_sim)
                    else:
                        cosine_scores[method].append(0.0)
                else:
                    cosine_scores[method].append(0.0)

        except Exception as e:
            print(f"Ошибка в паре {i+1}: {e}")
            for method in methods:
                flags[method].append(False)
                results[method].append({"ru_tokens": r_tokens, "en_tokens": e_tokens, "alignment": []})
                cosine_scores[method].append(0.0)

    if output_file:
        for method in methods:
            total_aligned = sum(flags[method])
            avg_cosine = np.mean([score for score in cosine_scores[method] if score > 0]) if any(score > 0 for score in cosine_scores[method]) else 0.0

            f_out.write(f"\nМетод {method}:\n")
            f_out.write(f"Выравниваний: {total_aligned}/{len(rus_sents)}\n")
            f_out.write(f"Средняя косинусная схожесть: {avg_cosine:.4f}\n\n")

            f_out.write(f"Примеры для {method} (первые 20):\n")
            count = 0
            for i, item in enumerate(results[method]):
                if flags[method][i]:
                    f_out.write(f"#Пара {i+1}:\n")
                    f_out.write(f"RU: {rus_sents[i]}\n")
                    f_out.write(f"EN: {eng_sents[i]}\n")
                    f_out.write(f"Выравнивание: {item['alignment']}\n")
                    f_out.write(f"Косинусная схожесть: {cosine_scores[method][i]:.4f}\n\n")
                    count += 1
                    if count >= 20:
                        break

        f_out.close()
        print(f"Результаты сохранены в файл {output_file}")

    return flags, results, cosine_scores

def main():
    print("Чтение файлов...")
    ru_text = read_text(RU_FILE)
    en_text = read_text(EN_FILE)

    print("Токенизация...")
    rus_sents = tokenize_ru(ru_text)
    eng_sents = tokenize_en(en_text)

    rus_sents = [s for s in rus_sents if s.strip()]
    eng_sents = [s for s in eng_sents if s.strip()]
    n = min(len(rus_sents), len(eng_sents))
    rus_sents = rus_sents[:n]
    eng_sents = eng_sents[:n]

    print(f"Всего предложений: {n}")

    print("Запуск выравнивания...")
    try:
        flags, aligned, cosine_scores = run_simalign(rus_sents, eng_sents, output_file=OUTPUT_FILE)
        
        for method in flags.keys():
            total_aligned = sum(flags[method])
            avg_cosine = np.mean([score for score in cosine_scores[method] if score > 0]) if any(score > 0 for score in cosine_scores[method]) else 0.0
            
            print(f"\nМетод {method}:")
            print(f"Выравниваний: {total_aligned}/{n}")
            print(f"Средняя косинусная схожесть: {avg_cosine:.4f}")
            
            print(f"\nПримеры для {method} (первые 5):")
            count = 0
            for i, item in enumerate(aligned[method]):
                if flags[method][i]:
                    print(f"#Пара {i+1}:")
                    print("RU:", rus_sents[i])
                    print("EN:", eng_sents[i])
                    print("Выравнивание:", item['alignment'])
                    print(f"Косинусная схожесть: {cosine_scores[method][i]:.4f}\n")
                    count += 1
                    if count >= 5:
                        break
    except Exception as e:
        print(f"Ошибка SimAlign: {e}")

if __name__ == "__main__":
    main()

Чтение файлов...
Токенизация...
Всего предложений: 1064
Запуск выравнивания...
Инициализация SimAlign...
Ошибка при создании aligner: 'inter'
Попытка создания с настройками по умолчанию...


2025-05-17 02:10:01,906 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: distilbert-base-multilingual-cased


SimAlign готов!
Обработка 1064 предложений...
Обработано 0/1064 предложений...
Обработано 50/1064 предложений...
Обработано 100/1064 предложений...
Обработано 150/1064 предложений...
Обработано 200/1064 предложений...
Обработано 250/1064 предложений...
Обработано 300/1064 предложений...
Обработано 350/1064 предложений...
Обработано 400/1064 предложений...
Обработано 450/1064 предложений...
Обработано 500/1064 предложений...
Обработано 550/1064 предложений...
Обработано 600/1064 предложений...
Обработано 650/1064 предложений...
Обработано 700/1064 предложений...
Обработано 750/1064 предложений...
Обработано 800/1064 предложений...
Обработано 850/1064 предложений...
Обработано 900/1064 предложений...
Обработано 950/1064 предложений...
Обработано 1000/1064 предложений...
Обработано 1050/1064 предложений...
Результаты сохранены в файл alignment_results.txt

Метод inter:
Выравниваний: 1064/1064
Средняя косинусная схожесть: 0.5904

Примеры для inter (первые 5):
#Пара 1:
RU: Ночь первая




Б